In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import pytraj as pt
import nglview as nv

import matplotlib
from matplotlib import pyplot as plt

import cphmdanalysis as cphmda

In [55]:
nRuns=1
sysName='WT'
with open('pH_List.txt','r') as pHlistFile:
    rep_pH_vals=list(map(float,pHlistFile.readline().split(', ')))
phList=rep_pH_vals*nRuns
nReps=len(rep_pH_vals)
lambdaFileNames=['%s.rep%02g.%02g.lambda'%(sysName,irep+1,irun+1) \
                 for irun in np.arange(nRuns) \
                 for irep in np.arange(len(rep_pH_vals))]
lambdaFiles=[cphmda.lambda_file(filename) for filename in lambdaFileNames]

for lambda_file in lambdaFiles:
    lambda_file.compute_s_values()

pka_data=cphmda.compute_pkas(phList,lambdaFiles) #wrapped_lambda_files)

tempStruc=pt.load('%s.min.rst7'%sysName,top='%s.parm7'%sysName)
pKaTable=pd.DataFrame([(tempStruc.top.residue(res-1).name,
 (tempStruc.top.residue(res-1)).original_resid,
  "%.3f"%(
      pka_data[ires][2]),
  "%.3f"%(
      pka_data[ires][0])) for ires,res in enumerate(lambdaFiles[0].titrreses)],
            columns=['ResName','Resid','pKa','Hill_Coef'])
pKaTable.to_csv('pKa_DataTable.csv',index=False)
goodRows=np.sum((pKaTable.pKa!='nan'))
totalRows=pKaTable.shape[0]
print('Total Number of Residues: ',
      totalRows,
      '\nNumber of succesfully titrated residues: ',
      goodRows,
      '\nNumber of unsuccesfully titrated residues: ',
      totalRows-goodRows)
pKaTable.head()

Total Number of Residues:  95 
Number of succesfully titrated residues:  68 
Number of unsuccesfully titrated residues:  27


,ResName,Resid,pKa,Hill_Coef
0,AS2,6,2.526,1.152
1,AS2,11,3.224,0.534
2,AS2,14,3.842,1.762
3,HIP,15,6.672,0.804
4,CYS,17,10.587,0.401
